# Seawater intrusion

In coastal aquifers, fresh groundwater (derived from terrestrial recharge) interacts with seawater. The location where the fresh groundwater and the seawater interact is commonly referred to as the interface. As seawater is denser than freshwater, the seawater tends to form a wedge under the freshwater. A conceptual model of an unconfined coastal aquifer is shown in Figure 1. 

<img src="SWI_fig_1.png" width="400"/>

**Figure 1. Description of hydrogeological variables for an unconfined coastal aquifer (from Morgan et al. 2013).**

The water budget involves net recharge to the aquifer $ W_{net} $ [L<sup>2</sup>/T]  (which accounts for infiltration, evapotranspiration and distributed pumping), flow into the aquifer through the inland boundary $ q_{b} $ [L<sup>2</sup>/T] and the flow at the coast $ q_{0} $ [L<sup>2</sup>/T]. 

The hydraulic head $ h_f $ [L] is related to the depth of the interface $ z $ [L] by the Ghyben-Herzberg relation $ z = h_f / δ $. Where $ δ $ [-] is the dimensionless density ratio, and $ ρ_s $ (= 1025 kg/m<sup>3</sup>) and $ ρ_f $ (= 1000 kg/m<sup>3</sup>) are freshwater and seawater densities, respectively [M/L<sup>3</sup>], so that $ δ $ = 0.025. 

 The freshwater thickness is $ h $ [L], and the base of the aquifer is $ z_0 $ [L] below mean sea level. Zone 1 is the region inland of the saltwater wedge and Zone 2 is the region between the coast and the inland extent ($ x_T $) of the saltwater wedge. $ x_T $ is commonly referred to as the wedge toe.

We can use equations first presented by Strack (1976) and further described and developed by Werner et al. (2012), Morgan et al. (2013) and Morgan and Werner (2015) to estimate water budget components, water table height and the interface position in a coastal aquifer.

The equation for the groundwater flux at the coastline in Zone 1 is:

$$
  q_o =\frac{K((h_b+z_0)^2 - (1 + \delta)z_0^2)+W_{net}x_b^2}{2x_b}\qquad{(\text{Equation 1})}             
$$ 

And in Zone 2 it is:

$$
  q_o =\frac{1+\delta}{\delta}\frac{K}{2x_b}h_b^2+\frac{W_{net}x_b}{2}\qquad{(\text{Equation 2})}             
$$ 

Here, $ K $ is the hydraulic conductivity [L/T], $ h_b $ [L] is a measured hydraulic head a distance $ x_b $ [L] from the coast, and other parameters are defined above.


Note that the choice of equation (1) or (2) for estimating $ q_0 $ depends on whether the measured $ h_b $ value occurs within Zone 1 (i.e., head measurement is inland of the interface) or Zone 2 (i.e., head measurement is above the interface). The location of the wedge toe $ x_T $ [L] is derived through considering that $ h_f = δz_0 $ at the wedge toe (can you see how this has been derived from the Ghyben-Herzberg equation?). 


Let's start by calculating the value of $ q_0 $ for an aquifer with the following properties:
$ h_b = 3 $ m, $ x_b = 3500 $ m, $ K = 10 $ m/d, $ W_{net} = 20 $ mm/yr, $ z_o = 20 $ m, $ \delta = 0.025 $.

In [58]:
import numpy as np
from matplotlib import pyplot as plt
from ipywidgets import interact, FloatSlider
%matplotlib inline

In [59]:
# all units in m and d
hb = 3.
xb = 3500.
K = 10.
Wnet = 20.*1.e-3/365.25  # convert to m/d
z0 = 20.
delta = 0.025

# determine whether observed data is in zone 1 or 2 and calculate q0
if hb < z0 * delta:
    print("Your observation data is in zone 2")
else:
    print("Your observation data is in zone 1")
    
# function to calculate q0 flux at the coast
def q0f(K,hb,z0,Wnet,xb,delta=0.025):   
    if hb < z0 * delta:
        return ((1+delta)/delta)*K/(2*xb)*hb**2+Wnet*xb/2
    else:
        return (K*((hb+z0)**2-(1+delta)*z0**2)+Wnet*xb**2)/(2*xb)  

q0 = q0f(K,hb,z0,Wnet,xb)
txt = "q\u2080 = {:.3f} m\u00b2/d "    #Here using unicode for subscript and superscript

print(txt.format(q0))

Your observation data is in zone 1
q₀ = 0.266 m²/d 


Once we have the value of $ q_0 $, we can estimate the water table distribution, which in Zone 1 is given by:

$$
  h_f =\sqrt{\frac{2q_0x-W_{net}x^2}{K}+(1+\delta)z_0^2}-z_0\qquad{(\text{Equation 3})}             
$$

And in Zone 2 it is:

$$
  h_f =\sqrt{(\frac{\delta}{1+\delta})\frac{2q_0x-W_{net}x^2}{K}}\qquad{(\text{Equation 4})}             
$$

We can also now estimate the interface position in Zone 1 and Zone 2 using the Ghyben Herzberg relation $ z = h_f /δ $. 

And we can calculate the wedge toe position $ x_t $ by:

$$
  x_t =\frac{q_0}{W_{net}}-\sqrt{(\frac{q_0}{W_{net}})^2-\frac{K\delta(1+\delta)z_0^2}{W_{net}}}\qquad{(\text{Equation 5})}             
$$

And while the wedge toe is a really useful indicator of inland seawater extent, another is the volume of seawater $ V_{sw} $ [L<sup>3</sup>/L/T] per metre of coastline: 

$$
V_{sw} = nz_0 (x_t-\frac{x_n}{2}(\sqrt{\frac{1}{M}}\arcsin\sqrt{M}-\sqrt{1-M}))\qquad{(\text{Equation 6})}
$$

Where $n$ [-] is porosity, $x_n = q_0/ W_{net}$ and $M$ = $K \delta (1+ \delta) z_0^2 / (W_{net} x_n^2)$


Let's pull this all together to plot the water table and interface for the coastal aquifer settings given above.

In [60]:
# all units in m and d
hb = 3.
xb = 3500.
K = 10.
Wnet = 20.*1.e-3/365.25  # convert to m/d
z0 = 20.
delta = 0.025

# function to calculate q0 flux at the coast
def q0f(K,hb,z0,Wnet,xb,delta=0.025):   
    if hb < z0 * delta:
        return ((1+delta)/delta)*K/(2*xb)*hb**2+Wnet*xb/2
    else:
        return (K*((hb+z0)**2-(1+delta)*z0**2)+Wnet*xb**2)/(2*xb)  

q0 = q0f(K,hb,z0,Wnet,xb)

# function to calculate toe position
def xtf(q0,Wnet,K,z0,delta=0.025):
    return q0/Wnet-np.sqrt((q0/Wnet)**2-(K*delta*(1+delta)*z0**2)/Wnet)

xt = xtf(q0,Wnet,K,z0,delta=0.025)

'''
# calculate water table David's method
def hff(x, q0, Wnet, K, z0, xt, delta=0.025):
    try:
        return np.array([hff(xi, q0, Wnet, K, z0, xt, delta=0.025) for xi in x])
    except TypeError:
        if x > xt:
            return np.sqrt((2*q0*x-Wnet*x**2)/K+(1+delta)*z0**2) - z0
        elif x < 0:
            return 0.
        else:
            return np.sqrt((delta/(1+delta))*(2*q0*x-Wnet*x**2)/K)
       
 '''
#calculate water table 
def hff(x, q0, Wnet, K, z0, xt, delta=0.025):
    hf = np.array([])  
    for xi in x:
        if xi > xt:
            hf = np.append(hf, np.sqrt((2*q0*xi-Wnet*xi**2)/K+(1+delta)*z0**2) - z0)
        elif xi < 0:
            hf = np.append(hf,0.)
        else:
            hf = np.append(hf,np.sqrt((delta/(1+delta))*(2*q0*xi-Wnet*xi**2)/K))
    return hf

#calculate Vsw
def Vswf(xt,q0,Wnet,K,z0,delta=0.025,n=0.3):
    xn = (q0/Wnet)
    M = (K  *delta*(1+ delta)*z0**2)/(Wnet*(q0/Wnet)**2)
    return (n*z0*(xt-xn/2*(np.sqrt(1/M)*np.arcsin(np.sqrt(M))-np.sqrt(1-M))))
    
#create interface function
def interface(K, Wnet, hb):
    # set the parameters
    Wnet *= 1.e-3/365.25
    z0 = 20.
    xb = 3500.
    delta = 0.025
    xlim = [-100,500]
    x = np.linspace(*xlim, 1001)
    
    # calculate ground surface as 3 x water table for these numbers
    h0 = 3*hff(x, q0f(20,3,z0,20*1.e-3/365.25,xb), 20*1.e-3/365.25, 20, z0, xtf(q0f(20,3,z0,20*1.e-3/365.25,xb),20*1.e-3/365.25,20,z0))
    
    q0 = q0f(K,hb,z0,Wnet,xb)  # calc outflow
    xt = xtf(q0,Wnet,K,z0)   # calc toe
    hf = hff(x, q0, Wnet, K, z0, xt)   # calc water table
    Vsw = Vswf(xt,q0,Wnet,K,z0,delta=0.025,n=0.3)
    

    f,ax = plt.subplots(1,1,figsize=(12,6))
    z = -hf/delta  # calc interface
    z = np.array([np.max([-z0, zi]) for zi in z])  # truncate at aquifer base
    ax.fill_between(x, 0.*z-z0, h0, color='#e1c699', alpha=0.75)  # shade land
    ax.fill_between(x, z, hf, color='b', alpha=0.5)    # shade freshwater
    ax.fill_between(x, 0.*z-z0, z, color='b', alpha=0.7)   # shade saltwater
    ax.plot(x,hf,'k-')  # water table
    ax.plot(x,z,'k-')  # interface
    ax.axhline(0, color='k', linestyle='--') # horizontal line at sea level
    ax.axvline(xt, color='k', linestyle='--') # vertical line dividing zones 1 and 2
    ax.set_xlim(xlim[::-1])    # reverses the x-axis direction
    ax.set_ylim([-z0-0.5, 3.])   # fix ylim to avoid replotting when new parameters
    # add labels for key variables
    ax.text(0.98,0.99,'$q_0$={:3.2f} m$^2$/d'.format(q0), size=12, ha='right', va='top', transform=ax.transAxes)
    ax.text(0.98,0.94,'$x_t$={:d} m'.format(int(xt)), size=12, ha='right', va='top', transform=ax.transAxes)
    ax.text(0.98,0.81,'$V\u209B_w$={:d} m$^2$/d'.format(int(Vsw)), size=12, ha='right', va='top', transform=ax.transAxes)    
    plt.show()

# slider objects for varying parameters in argument of interface() function
K = FloatSlider(description='K (m/d)',min=1,max=100,step=1,value=10)
Wnet = FloatSlider(description='W (mm/y)',min=0.1,max=100,step=0.1,value=20)
hb = FloatSlider(description='hb (m)',min=0.2,max=10,step=0.1,value=3)

# sets up the interactive widget
interact(interface, K=K, Wnet=Wnet, hb=hb)
    

        

interactive(children=(FloatSlider(value=10.0, description='K (m/d)', min=1.0, step=1.0), FloatSlider(value=20.…

<function __main__.interface(K, Wnet, hb)>

**Q1. How does changing the value of $ h_b $ influence $ q_0 $? Why is this?**

**Q2. Explore the types of coastal unconfined aquifers (i.e., high versus low $ K $, high versus low $ W_{net} $, high versus low $ x_b $) that will have largest inland seawater extent, under natural conditions?**

## References


Morgan, L.K., Werner, A.D., Morris, M.J., Teubner, M., 2013. Application of a Rapid-Assessment Method for Seawater Intrusion Vulnerability: Willunga Basin, South Australia. In Wetzelhuetter C (Ed.) Groundwater in the Coastal Zones of Asia-Pacific: 205-225.Springer Science & Business Media. http://dx.doi.org/10.1007/978-94-007-5648-9_10.

Morgan, L.K., Werner, A.D., 2015. A national inventory of seawater intrusion vulnerability for Australia. Journal of Hydrology: Regional Studies 4: 686-698. http://dx.doi.org/10.1016/j.ejrh.2015.10.005.

Strack, O.D.L., 1976. Single-potential solution for regional interface problems in coastal aquifers, Water Resources Research 12: 1165-1174.

Werner, A.D., Ward, J.D., Morgan, L.K., Simmons, C.T., Robinson, N.I., Teubner, M.D., 2012. Vulnerability indicators of seawater intrusion. Ground Water 50(1): 48-58.
